In [1]:
import torch
import torch.nn as nn
from torch import optim

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [2]:
import numpy as np
import sys
import os
#import tensorflow as tf

import io
from gensim.models import word2vec

from nltk.tokenize import word_tokenize
import nltk

WV_PARAMETER_ROOT = "/home/jongsu/jupyter/pytorch_dialogue_ie/parameter/dialogue_wv"
RNN_PARAMETER_ROOT = '/home/jongsu/jupyter/pytorch_dialogue_ie/parameter/character_rnn.pth'

In [3]:
def makeinput(word):
  newtensor = torch.tensor(np.zeros(shape = (len(word), 1, 100), dtype="float32")).cuda()
  i = 0
  while(i < len(word)):
    
    newtensor[i][0][word[i]] = 1
    i = i + 1
  return newtensor  

In [4]:
class chr_rnn(nn.Module):
    def __init__(self, hidden_size, bidirectional):
        super(chr_rnn, self).__init__()
        self.hidden_size = hidden_size
        self.lastnet = nn.Linear(200,100)
        if(bidirectional == True):
            self.bidirectional = 2
        else:
            self.bidirectional = 1
        self.chr_rnn = nn.GRU(100, 100, bidirectional = bidirectional)
        
    def forward(self,char,h0):
        _,h0 = self.chr_rnn(char,h0)
        h0 = h0.view(-1,200)
        last_w = self.lastnet(h0)
        
        return last_w
        
    def initHidden(self):
        return torch.zeros(self.bidirectional , 1, self.hidden_size, device=device)
    
    
chr_rnn_1 = chr_rnn(100, True).cuda()

chr_optimizer = optim.SGD(chr_rnn_1.parameters(), lr=0.01)



criterion = nn.MSELoss()


In [5]:
wv_model_en = word2vec.Word2Vec(size=100, window=5, min_count=5, workers=4)
wv_model_en = word2vec.Word2Vec.load(WV_PARAMETER_ROOT)

In [6]:
if os.path.isfile(RNN_PARAMETER_ROOT):
    print("loading chr")
    chr_rnn_1.load_state_dict(torch.load(RNN_PARAMETER_ROOT))
    

loading chr


In [7]:
#make character based wv
iter = 0



while(iter < 50): #train 500 times

    i = 0
    loss = 0
    chr_optimizer.zero_grad()

    totalloss = 0
    for word, vocab_obj in wv_model_en.wv.vocab.items(): #all dataset = 251 * 25

        p = 0
        h0 = chr_rnn_1.initHidden()
        #if(i > 30):
        #    continue
        word_ = []
        j = 0
        while(j < len(word)):
          word_.append(ord(word[j])%100)
          j = j + 1
          
        gru_input = makeinput(word_)
        h0 = chr_rnn_1(gru_input, h0)
            
        
        wv_value = torch.from_numpy(wv_model_en.wv[word]).cuda()
        
        loss = loss + criterion(h0.view(-1,1),wv_value.view(-1,1))
        
        
        
        if i % 251 == 250: #batch size = 251
            print(loss)
            chr_optimizer.zero_grad()
            loss.backward()
            chr_optimizer.step()
            loss = 0

        i = i + 1


    print(totalloss)
    print("iter --------------------------------------",iter)
    iter = iter + 1

    if iter%10 == 0:
        print(wv_value)
        #print(list(chr_rnn.parameters()))

    torch.save(chr_rnn_1.state_dict(),RNN_PARAMETER_ROOT)
print("a")



tensor(259.9157, device='cuda:0')
tensor(212.3062, device='cuda:0')
tensor(228.5942, device='cuda:0')
tensor(254.7605, device='cuda:0')
tensor(245.2538, device='cuda:0')
tensor(242.1407, device='cuda:0')
tensor(231.2613, device='cuda:0')
tensor(232.7893, device='cuda:0')
tensor(205.1482, device='cuda:0')
tensor(211.6225, device='cuda:0')
tensor(246.9273, device='cuda:0')
tensor(211.7027, device='cuda:0')


KeyboardInterrupt: 